#### First Implementation of scraping all events of defined Website Scrapers and Visualize them in Qgrid

In [ ]:
import datetime as dt
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Button, Dropdown, FloatText, BoundedFloatText, BoundedIntText, Label, ToggleButtons, HTML, Checkbox, AppLayout, Layout
import webbrowser
from src import scraper_collection

In [ ]:
label_layout  = {'width': '230px'}
button_layout = {'width': '100px'}
widget_list = [{
    'label': Label(scraper.name, layout = label_layout),
    'start_scraper': Button(description='get events', layout = button_layout),
    'open_url': Button(description='open url', layout = button_layout)
} for scraper in scraper_collection.scrapers.values()]

HBox_list = [HBox([widget for widget in widget_dict.values()]) for widget_dict in widget_list]
app = VBox(HBox_list)

def open_url_func(url):
    def open_url(button):
        webbrowser.open_new_tab(url)
    return open_url

for i, widget_dict in enumerate(widget_list):
    button = widget_dict['open_url']
    button.on_click(open_url_func(list(scraper_collection.scrapers.values())[i].url))

app

In [ ]:
end_date= dt.datetime(2023,8,31)
print(end_date)
events = scraper_collection.get_all_events_df(end_date)

In [ ]:
from ipyaggrid import Grid

In [ ]:
cell_renderer_checkbox = '''function(params) { 
    var input = document.createElement('input');
    input.type="checkbox";
    input.checked=params.value;
    input.addEventListener('click', function (event) {
      params.value=!params.value;
      params.node.data.fieldName = params.value;
    });
    return input;
  }
'''

date_picker = '''function DatePicker() {}
DatePicker.prototype.init = function(params) {
  this.eInput = document.createElement('input');
  this.eInput.type = 'date';
  this.eInput.value = params.value;
  this.params = params;
  var that = this;
  this.eInput.addEventListener('input', function(event) {
    that.params.setValue(event.target.value);
  });
};
DatePicker.prototype.getGui = function() {
  return this.eInput;
};
'''
column_defs = [{'headerName':'Start','field':'start_date', 'cellDataType': 'dateString', 'editable': True},
               {'headerName':'Typ','field':'event_type', 'editable': True},
               {'headerName':'Titel','field':'title'},
               {'headerName':'URL','field':'url'},
               {'headerName':'Organisation','field':'organiser'},
               {'headerName':'Ort','field':'location'},
               {'headerName':'regelmäßig','field':'periodic', 'editable': True, 'cellDataType': 'boolean'},
              ]

grid_options = {
    'columnDefs' : column_defs,
    'enableSorting': True,
    'enableFilter': True,
    'enableColResize': True,
    'enableRangeSelection': True,
}

g = Grid(grid_data=events,
         grid_options=grid_options,
         theme='ag-theme-balham',
         quick_filter=True,
         show_toggle_edit=True,
         # export_mode="buttons",
         export_csv=True,
         export_excel=True,
         # show_toggle_delete=True,
         sync_on_edit=True,
         columns_fit='auto',
         index=False,
         keep_multiindex=False,)
g

In [ ]:
g.grid_data_out.get('grid')